## 1. Libraries and settings

In [ ]:
import numpy as np
import random
import pandas as pd 
from pylab import mpl, plt
plt.style.use('seaborn-v0_8-darkgrid')
mpl.rcParams['font.family'] = 'serif'
%matplotlib inline
import torch
import math, time
import itertools
import datetime
from operator import itemgetter
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
import torch
import torch.nn as nn
from torch.autograd import Variable

import vectorbtpro as vbt

from datetime import date

import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

import warnings
warnings.simplefilter("ignore", UserWarning)

vbt.settings.set_theme('dark')
vbt.settings['plotting']['layout']['width'] = 800
vbt.settings['plotting']['layout']['height'] = 400

# #hparams
timestep = 20
# # Update these dimensions based on your dataset

# hidden_dim = 32
# num_layers = 2

# num_epochs = 200
# learning_rate=0.01
# step_size=30
# gamma=0.9

# dropout_rate=0.2
# print_epochs = 2

In [ ]:
df = pd.read_csv('2ySOLdata1h.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
df.set_index('timestamp', inplace=True)

df['signal'] = df['signal'].replace({'SignalNone': 1, 'SignalLong': 2, 'SignalShort': 0})
df.fillna(method='pad');


In [ ]:
data = vbt.Data.from_data(df)
features = data.run("talib", mavp=vbt.run_arg_dict(periods=14))
data.data['symbol'] = pd.concat([data.data['symbol'], features], axis=1)
data.data['symbol'].drop(['Open', 'High', 'Low'], axis=1, inplace=True)
# This will drop columns from the DataFrame where all values are NaN
data.data['symbol'] = data.data['symbol'].dropna(axis=1, how='all')

open_price = data.get('Open')
high_price = data.get('High')
low_price = data.get('Low')
close_price = data.get('Close')

data.data['symbol'] = data.data['symbol'].dropna()
predictor_list = data.data['symbol'].drop('signal', axis=1).columns.tolist()


X = data.data['symbol'][predictor_list]

y = data.data['symbol']['signal']

X.columns = X.columns.astype(str)



In [ ]:
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


# First, split your data into a training+validation set and a separate test set
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.3)

# Then, split the training+validation set into a training set and a validation set
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2)  # 0.2 here means 20% of the original data, or 25% of the training+validation set

scaler = StandardScaler()


X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_val_scaled = scaler.fit_transform(X_val)


In [ ]:
def create_sequences(input_data, sequence_length):
    sequences = []
    data_len = len(input_data)
    for i in range(data_len - sequence_length):
        seq = input_data[i:(i + sequence_length)]
        sequences.append(seq)
    return np.array(sequences)

In [ ]:
X_train_list = create_sequences(X_train_scaled, timestep)
X_test_list = create_sequences(X_test_scaled, timestep)
X_val_list = create_sequences(X_val_scaled, timestep)

In [ ]:
y_train_seq_ar = y_train[timestep:]
y_test_seq_ar = y_test[timestep:]
y_val_seq_ar = y_val[timestep:]

In [ ]:
x_train_ar = np.array(X_train_list)
x_test_ar = np.array(X_test_list)  
x_val_ar = np.array(X_val_list)  

y_train_seq = np.array(y_train_seq_ar)
y_test_seq = np.array(y_test_seq_ar)
y_val_seq = np.array(y_val_seq_ar)

In [ ]:
# Check for MPS (GPU on M1 Mac) availability and set it as the device
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:

# Convert to tensors
X_train_tensor = torch.tensor(x_train_ar, dtype=torch.float32) # .to(device)
y_train_tensor = torch.tensor(y_train_seq, dtype=torch.long)
X_test_tensor = torch.tensor(x_test_ar, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_seq, dtype=torch.long)
X_val_tensor = torch.tensor(x_val_ar, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val_seq, dtype=torch.long)


In [ ]:

from sklearn.utils.class_weight import compute_class_weight

# Convert y_train to a numpy array if it's a tensor

if isinstance(y_train_seq, torch.Tensor):
    y_train_seq_np = y_train_seq.cpu().numpy()
else:
    y_train_seq_np = y_train_seq  # Assuming y_train_seq is already a numpy array or similar

class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_seq_np), y=y_train_seq_np)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)


# Move class weights to the same device as your model and data
class_weights_tensor = class_weights_tensor.to(device)  # device could be 'cpu' or 'cuda'


In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
import optuna
import numpy as np

# Assuming X_train_tensor and y_train_tensor are your input and output training tensors
# Make sure X_train_tensor and y_test_tensor are already tensors

# Check for MPS (GPU on M1 Mac) availability and set it as the device
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

num_epochs = 100
input_dim = 175  # Number of features
output_dim = 3  # Number of classes


class BiLSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim, dropout_rate):
        super(BiLSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        # Bidirectional LSTM Layer
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, bidirectional=True)
        
        # Dropout layer
        self.dropout = nn.Dropout(dropout_rate)
        
        # Fully connected layer
        # The input dimension is twice the hidden_dim because it's bidirectional
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, x):
        batch_size = x.size(0)
        # Initialize hidden state and cell state
        h0 = torch.zeros(self.num_layers * 2, batch_size, self.hidden_dim).to(device)
        c0 = torch.zeros(self.num_layers * 2, batch_size, self.hidden_dim).to(device)
        
        # Forward propagate LSTM
        out, (hn, cn) = self.lstm(x, (h0, c0))
        
        # Concatenate the hidden states from both directions
        out = torch.cat((hn[-2,:,:], hn[-1,:,:]), dim = 1)
        
        # Pass the concatenated hidden states to the fully connected layer
        out = self.fc(out)
        
        return out

def objective(trial):
    # Suggest hyperparameters
    hidden_dim = trial.suggest_categorical('hidden_dim', [16, 32, 64])
    num_layers = trial.suggest_int('num_layers', 1, 3)
    lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)
    step_size = trial.suggest_int('step_size', 10, 100)
    gamma = trial.suggest_float('gamma', 0.85, 0.99)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.4)

    # Initialize model and move it to the MPS device
    model = BiLSTMClassifier(input_dim=X_train_tensor.shape[2], hidden_dim=hidden_dim, num_layers=num_layers, output_dim=len(np.unique(y_train_tensor.cpu().numpy())), dropout_rate=dropout_rate).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)
    criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)

    X_train_tensor_gpu = X_train_tensor.to(device)
    y_train_tensor_gpu = y_train_tensor.to(device)
    X_val_tensor_gpu = X_val_tensor.to(device)
    y_val_tensor_gpu = y_val_tensor.to(device)
    X_test_tensor_gpu = X_test_tensor.to(device)
    y_test_tensor_gpu = y_test_tensor.to(device)

    # Training loop
    model.train()
    for epoch in range(num_epochs):  # use a small number of epochs for demonstration
        optimizer.zero_grad()
        output = model(X_train_tensor_gpu)
        loss = criterion(output, y_train_tensor_gpu)
        loss.backward()
        optimizer.step()
        scheduler.step()

        # Validation step
        if epoch % 10 == 0:
            model.eval()
            with torch.no_grad():
                val_output = model(X_val_tensor_gpu)
                val_loss = criterion(val_output, y_val_tensor_gpu)
            model.train()

    # model.eval()
    # with torch.no_grad(): 
        # predictions = model(X_test_tensor_gpu)
        # test_loss = criterion(predictions, y_test_tensor_gpu)
        
    # return test_loss.item()
    
    return val_loss.item()

# Before running the study, ensure your data tensors are on the CPU as Optuna will handle moving them to the GPU
X_train_tensor = X_train_tensor.cpu()
y_train_tensor = y_train_tensor.cpu()
X_val_tensor = X_val_tensor.cpu()
y_val_tensor = y_val_tensor.cpu()
X_test_tensor = X_test_tensor.cpu()
y_test_tensor = y_test_tensor.cpu()

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)

print('Best trial:', study.best_trial.params)


[I 2024-02-24 15:32:16,551] A new study created in memory with name: no-name-f2d634c2-651f-46d4-8de1-4f13cba7830d


Using device: mps


[I 2024-02-24 15:33:19,643] Trial 0 finished with value: 3.855517625808716 and parameters: {'hidden_dim': 64, 'num_layers': 3, 'lr': 0.0008057052762749867, 'step_size': 30, 'gamma': 0.9820801163811199, 'dropout_rate': 0.1439361557306276}. Best is trial 0 with value: 3.855517625808716.
[I 2024-02-24 15:33:32,066] Trial 1 finished with value: 7.601255416870117 and parameters: {'hidden_dim': 32, 'num_layers': 1, 'lr': 0.03742393108033467, 'step_size': 18, 'gamma': 0.8651227660612031, 'dropout_rate': 0.14385066918291542}. Best is trial 0 with value: 3.855517625808716.
[I 2024-02-24 15:33:55,240] Trial 2 finished with value: 8.367830276489258 and parameters: {'hidden_dim': 64, 'num_layers': 1, 'lr': 0.021046257584648502, 'step_size': 55, 'gamma': 0.9891089814163857, 'dropout_rate': 0.16095509009397027}. Best is trial 0 with value: 3.855517625808716.
[I 2024-02-24 15:34:07,058] Trial 3 finished with value: 1.1158785820007324 and parameters: {'hidden_dim': 32, 'num_layers': 1, 'lr': 5.8732761

Best trial: {'hidden_dim': 64, 'num_layers': 3, 'lr': 2.21015078595355e-05, 'step_size': 62, 'gamma': 0.9877575829732127, 'dropout_rate': 0.21386160007020788}


In [ ]:
# # Adjust the figure size
# plt.figure(figsize=(6, 3))

# # Plot the training loss
# plt.plot(hist, label="Training loss")
# plt.legend()

# # Show the plot
# plt.show()

In [ ]:
# with torch.no_grad():
#     y_test_pred = model(x_test)
#     # Convert logits to probabilities
#     probabilities = torch.softmax(y_test_pred, dim=1)
#     # Get the predicted class labels
#     _, predicted_labels = torch.max(probabilities, 1)
#     # Move the tensor to CPU and then convert to numpy
#     predicted_labels_numpy = predicted_labels.cpu().numpy()
#     print(len(predicted_labels_numpy))


In [ ]:
# df_split = data.data['symbol'][-len(predicted_labels_numpy):].copy()
# df_split.loc[:, "signal"] = predicted_labels_numpy
# signal = df_split['signal']
# entries = signal == 2
# exits = signal == 0

In [ ]:
# pf = vbt.Portfolio.from_signals(
#     close=df_split.Close, 
#     long_entries=entries, 
#     long_exits=exits,
#     size=100,
#     size_type='value',
#     init_cash='auto'
# )

In [ ]:
# pf.plot(settings=dict(bm_returns=False)).show()

In [ ]:
# pf.stats()

In [ ]:
# from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# # Convert tensors to numpy arrays for use with Scikit-Learn
# true_labels = y_test.cpu().numpy()
# pred_labels = predicted_labels.cpu().numpy()

# precision = precision_score(true_labels, pred_labels, average='macro')  # 'macro' for unweighted mean
# recall = recall_score(true_labels, pred_labels, average='macro')
# f1 = f1_score(true_labels, pred_labels, average='macro')
# conf_matrix = confusion_matrix(true_labels, pred_labels)

# print(f'Precision: {precision:.2f}')
# print(f'Recall: {recall:.2f}')
# print(f'F1 Score: {f1:.2f}')
# print('Confusion Matrix:\n', conf_matrix)